In [ ]:
## Hybrid Search Langchain

In [1]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("PINECONE_API_KEY")

In [5]:

from pinecone import Pinecone, ServerlessSpec
index_name = "hybrid-search-langchain-pinecone"

## initialize the pinecone client
pc = Pinecone(api_key = api_key)

## create the index in pinecone
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        dimension = 384, #dimension of dense vector created using huggingface embedding
        metric = 'dotproduct', # sparse values supported only for dotproduct
        spec = ServerlessSpec(cloud='aws', region='us-east-1')
    )

In [6]:
index = pc.Index(index_name)
index

In [9]:
## creating vector embedding and sparse matrix
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [10]:
# for creating sparse matrix
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\praba\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\praba\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [11]:
sentences = [
    "In 2023, I visited Paris",
    "In 2022, I visited Berlin",
    "In 2021, I visited Kathmandu",
]

# tfidf values on these sentences 
bm25_encoder.fit(sentences)

# store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object 
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 409.15it/s]


In [12]:
# creating the retriever 
retriever = PineconeHybridSearchRetriever(
    embeddings = embeddings, 
    sparse_encoder = bm25_encoder, 
    index = index
    )

In [13]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000002ED77C87DF0>, index=<pinecone.data.index.Index object at 0x000002ED5F640A30>)

In [14]:
retriever.add_texts(sentences)
# now you can go to the pinecone index online to check if it has the vector embeddings as well as the texts

100%|██████████| 1/1 [00:01<00:00,  1.62s/it]


In [17]:
retriever.invoke("Which cities did I visit last?")

[Document(page_content='In 2023, I visited Paris'),
 Document(page_content='In 2022, I visited Berlin'),
 Document(page_content='In 2021, I visited Kathmandu')]